In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth=True  
sess = tf.Session(config=tf_config)

import sys
if not '../' in sys.path: sys.path.append('../')

import numpy as np
import pandas as pd

from utils import data_utils
from model_config import config
from ded_detAttn import DetSeq2SeqDetAttnModel

Using TensorFlow backend.


In [3]:
print('[INFO] Preparing data for experiment: {}'.format(config['experiment']))
if config['experiment'] == 'qgen':
    train_data = pd.read_csv(config['data_dir'] + 'df_qgen_train.csv')
    val_data = pd.read_csv(config['data_dir'] + 'df_qgen_val.csv')
    test_data = pd.read_csv(config['data_dir'] + 'df_qgen_test.csv')
    input_sentences = pd.concat([train_data['answer'], val_data['answer'], test_data['answer']])
    output_sentences = pd.concat([train_data['question'], val_data['question'], test_data['question']])
    true_test = test_data['question']
    input_test = test_data['answer']
    filters = '!"#$%&()*+,./:;<=>?@[\\]^`{|}~\t\n'
    w2v_path = config['w2v_dir'] + 'w2vmodel_qgen.pkl'
    
elif config['experiment'] == 'dialogue':
    train_data = pd.read_csv(config['data_dir'] + 'df_dialogue_train.csv')
    val_data = pd.read_csv(config['data_dir'] + 'df_dialogue_val.csv')
    test_data = pd.read_csv(config['data_dir'] + 'df_dialogue_test.csv')
    input_sentences = pd.concat([train_data['line'], val_data['line'], test_data['line']])
    output_sentences = pd.concat([train_data['reply'], val_data['reply'], test_data['reply']])
    true_test = test_data['reply']
    input_test = test_data['line']
    filters = '!"#$%&()*+/:;<=>@[\\]^`{|}~\t\n'
    w2v_path = config['w2v_dir'] + 'w2vmodel_dialogue.pkl'

else:
    print('Invalid experiment name specified!')
    


[INFO] Preparing data for experiment: qgen


In [4]:
print('[INFO] Tokenizing input and output sequences')
x, input_word_index = data_utils.tokenize_sequence(input_sentences, 
                                                                      filters, 
                                                                      config['encoder_num_tokens'], 
                                                                      config['encoder_vocab'])

y, output_word_index = data_utils.tokenize_sequence(output_sentences, 
                                                                        filters, 
                                                                        config['decoder_num_tokens'], 
                                                                        config['decoder_vocab'])

print('[INFO] Split data into train-validation-test sets')
x_train, y_train, x_val, y_val, x_test, y_test = data_utils.create_data_split(x, 
                                                                              y, 
                                                                              config['experiment'])

encoder_embeddings_matrix = data_utils.create_embedding_matrix(input_word_index, 
                                                               config['embedding_size'], 
                                                               w2v_path)

decoder_embeddings_matrix = data_utils.create_embedding_matrix(output_word_index, 
                                                               config['embedding_size'], 
                                                               w2v_path)

# Re-calculate the vocab size based on the word_idx dictionary
config['encoder_vocab'] = len(input_word_index)
config['decoder_vocab'] = len(output_word_index)

[INFO] Tokenizing input and output sequences
[INFO] Split data into train-validation-test sets


In [5]:
model = DetSeq2SeqDetAttnModel(config, 
                               encoder_embeddings_matrix, 
                               decoder_embeddings_matrix, 
                               input_word_index, 
                               output_word_index)

[INFO] Building Model ...


In [7]:
if config['load_checkpoint'] != 0: 
    checkpoint = 'models/det-seq2seq-det-attn-' + str(config['load_checkpoint']) + '.ckpt'
else:
    checkpoint = tf.train.get_checkpoint_state(os.path.dirname('models/checkpoint')).model_checkpoint_path

preds = model.predict(checkpoint, 
                      x_test, 
                      y_test, 
                      true_test, 
                      ['?'])

INFO:tensorflow:Restoring parameters from models/det-seq2seq-det-attn-3.ckpt
BLEU 1 to 4 : 30.5 | 14.07 | 7.97 | 4.64


In [8]:
model.show_output_sentences(preds[:10], 
                            y_test[:10], 
                            input_test[:10], 
                            true_test[:10], 
                            ['?'])

Input:      fantasia released as her coronation single i believe a song co written by season one finalist tamyra gray and degarmo released dreams
Actual:     what was the first song released by diana degarmo after coming in second on american idol ?
Generated: what is the name of the song that is released in the year ?

Input:      he increased the faculty by more than 500 professors the academic quality of the student body has improved dramatically with the average sat score rising from 1240 to 1360 the number of minority students more than doubled the endowment grew from 350 million to more than 3 billion the annual operating budget rose from 177 million to more than 650 million and annual research funding improved from 15 million to more than 70 million
Actual:     when malloy reached the end of his time as president how much annuals funding for research did notre dame have ?
Generated: what was the average score for the academic environment ?

Input:      together with the national